In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#this is a python magic function to automatically show matplotlib plots
%matplotlib inline 

### cleaning and formatting the Insurance data.removed state 'TN' and 'County' from column and dropped columns not required for
## representation of Data.Converted Numeric values to percentage.

In [2]:
TN_insured_data=pd.read_csv('../Data/TN_insurance_data.csv')

In [3]:
TN_insured_data.rename(columns = {'NIC_PT':'Insured', 'NUI_PT':'Not_Insured','NAME':'County','time':'Year'},inplace = True) # renamed column names
TN_insured_data=TN_insured_data.drop(columns = ['state','county']) ## dropped column not required


In [4]:
TN_insured_data['County'] = TN_insured_data['County'].str[:-11] ## removed State from County column
TN_insured_data

,Insured,Not_Insured,County,Year
0,52679,8602,Anderson,2010
1,30357,8542,Bedford,2010
2,10768,2333,Benton,2010
3,7957,1846,Bledsoe,2010
4,86269,15189,Blount,2010
...,...,...,...,...
1135,9664,1497,Wayne,2019
1136,22565,2938,Weakley,2019
1137,18547,2717,White,2019
1138,197201,13215,Williamson,2019


In [5]:
#calculated % of insured and not insured population
TN_insured_data['Percent_Insured']=round(TN_insured_data['Insured']/(TN_insured_data['Insured']+TN_insured_data['Not_Insured'])*100,2)
TN_insured_data['Percent_Uninsured']=round(TN_insured_data['Not_Insured']/(TN_insured_data['Insured']+TN_insured_data['Not_Insured'])*100,2)
TN_insured_data

,Insured,Not_Insured,County,Year,Percent_Insured,Percent_Uninsured
0,52679,8602,Anderson,2010,85.96,14.04
1,30357,8542,Bedford,2010,78.04,21.96
2,10768,2333,Benton,2010,82.19,17.81
3,7957,1846,Bledsoe,2010,81.17,18.83
4,86269,15189,Blount,2010,85.03,14.97
...,...,...,...,...,...,...
1135,9664,1497,Wayne,2019,86.59,13.41
1136,22565,2938,Weakley,2019,88.48,11.52
1137,18547,2717,White,2019,87.22,12.78
1138,197201,13215,Williamson,2019,93.72,6.28


### Merged it with FIPS County data to get FIPS code for each county to plot Map in Tablaeu 

In [6]:
##Downloaded FIPS data and filtered it for TN only
TN_FIPS=pd.read_excel('../Data/US_FIPS_Codes.xls',skiprows = 1)
TN_FIPS=TN_FIPS[TN_FIPS['State']=='Tennessee']
TN_FIPS.rename(columns = {'County Name':'County'},inplace = True)
TN_FIPS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 2425 to 2519
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   State      95 non-null     object
 1   County     95 non-null     object
 2   FIPS Code  95 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 3.0+ KB


In [19]:
## merged both the dataframe using left join to get FIPS code for only county in TN
TN_insured_data_1 = TN_insured_data.merge (TN_FIPS, how='left',on='County')
TN_insured_data_1=TN_insured_data_1.drop(columns = ['State'])
TN_insured_data_1


,Insured,Not_Insured,County,Year,Percent_Insured,Percent_Uninsured,FIPS Code
0,52679,8602,Anderson,2010,85.96,14.04,47001.0
1,30357,8542,Bedford,2010,78.04,21.96,47003.0
2,10768,2333,Benton,2010,82.19,17.81,47005.0
3,7957,1846,Bledsoe,2010,81.17,18.83,47007.0
4,86269,15189,Blount,2010,85.03,14.97,47009.0
...,...,...,...,...,...,...,...
1135,9664,1497,Wayne,2019,86.59,13.41,47181.0
1136,22565,2938,Weakley,2019,88.48,11.52,47183.0
1137,18547,2717,White,2019,87.22,12.78,47185.0
1138,197201,13215,Williamson,2019,93.72,6.28,47187.0


In [20]:
TN_insured_data_1.to_csv('TN_insured_data_1.csv')

In [9]:
#% of Insured population per county from 2010-2019
TN_not_insured_groups = TN_insured_data.groupby(['Year','County',])['Percent_Insured'].mean().unstack()
TN_not_insured_groups

County,Anderson,Bedford,Benton,Bledsoe,Blount,Bradley,Campbell,Cannon,Carroll,Carter,...,Unicoi,Union,Van Buren,Warren,Washington,Wayne,Weakley,White,Williamson,Wilson
Year,,,,,,,,,,,,,,,,,,,,,
2010,85.96,78.04,82.19,81.17,85.03,83.13,83.50,82.24,83.85,82.16,...,83.81,81.69,81.72,78.07,84.96,81.83,81.68,82.20,91.54,87.34
2011,84.59,79.41,81.40,78.79,83.85,81.69,83.45,83.00,83.87,81.96,...,82.53,81.34,81.19,78.88,82.90,80.40,82.18,81.95,90.78,86.20
2012,86.50,79.92,81.32,80.60,85.36,83.96,83.67,82.60,83.60,82.71,...,83.47,81.97,82.86,80.19,83.48,82.37,84.41,82.34,91.83,87.13
2013,86.53,78.21,81.80,79.13,85.17,82.25,83.22,83.35,83.97,82.25,...,83.63,82.05,81.63,79.51,84.54,81.44,84.12,83.18,91.12,86.95
2014,87.21,81.41,84.17,82.80,86.91,84.33,84.55,84.68,86.29,84.29,...,84.49,83.68,84.32,81.89,86.75,84.88,87.08,85.25,92.44,88.73
2015,89.84,83.17,85.53,85.34,89.10,87.11,87.44,87.17,88.74,86.06,...,86.69,84.67,87.69,85.02,88.51,87.52,89.70,86.56,94.01,90.73
2016,90.80,87.01,88.19,85.96,89.86,86.99,88.41,88.15,90.09,88.59,...,88.30,87.47,88.24,86.35,90.23,87.79,90.36,88.90,94.33,92.43
2017,89.72,85.60,87.60,84.92,89.75,86.65,87.57,88.01,89.14,87.99,...,87.67,87.05,87.18,85.19,88.77,88.16,89.70,88.63,93.71,91.38
2018,88.44,85.34,87.26,82.35,88.60,86.41,86.90,87.48,88.48,86.80,...,87.87,86.52,86.29,84.12,89.29,86.66,87.31,87.43,93.87,90.62


In [10]:
#% of UnInsured population per county from 2010-2019
TN_not_insured_groups = TN_insured_data.groupby(['Year','County',])['Percent_Uninsured'].mean().unstack()
TN_not_insured_groups

County,Anderson,Bedford,Benton,Bledsoe,Blount,Bradley,Campbell,Cannon,Carroll,Carter,...,Unicoi,Union,Van Buren,Warren,Washington,Wayne,Weakley,White,Williamson,Wilson
Year,,,,,,,,,,,,,,,,,,,,,
2010,14.04,21.96,17.81,18.83,14.97,16.87,16.50,17.76,16.15,17.84,...,16.19,18.31,18.28,21.93,15.04,18.17,18.32,17.80,8.46,12.66
2011,15.41,20.59,18.60,21.21,16.15,18.31,16.55,17.00,16.13,18.04,...,17.47,18.66,18.81,21.12,17.10,19.60,17.82,18.05,9.22,13.80
2012,13.50,20.08,18.68,19.40,14.64,16.04,16.33,17.40,16.40,17.29,...,16.53,18.03,17.14,19.81,16.52,17.63,15.59,17.66,8.17,12.87
2013,13.47,21.79,18.20,20.87,14.83,17.75,16.78,16.65,16.03,17.75,...,16.37,17.95,18.37,20.49,15.46,18.56,15.88,16.82,8.88,13.05
2014,12.79,18.59,15.83,17.20,13.09,15.67,15.45,15.32,13.71,15.71,...,15.51,16.32,15.68,18.11,13.25,15.12,12.92,14.75,7.56,11.27
2015,10.16,16.83,14.47,14.66,10.90,12.89,12.56,12.83,11.26,13.94,...,13.31,15.33,12.31,14.98,11.49,12.48,10.30,13.44,5.99,9.27
2016,9.20,12.99,11.81,14.04,10.14,13.01,11.59,11.85,9.91,11.41,...,11.70,12.53,11.76,13.65,9.77,12.21,9.64,11.10,5.67,7.57
2017,10.28,14.40,12.40,15.08,10.25,13.35,12.43,11.99,10.86,12.01,...,12.33,12.95,12.82,14.81,11.23,11.84,10.30,11.37,6.29,8.62
2018,11.56,14.66,12.74,17.65,11.40,13.59,13.10,12.52,11.52,13.20,...,12.13,13.48,13.71,15.88,10.71,13.34,12.69,12.57,6.13,9.38


#### Looked at the numbers for only WIlliamson County

In [11]:
#williamson county data only
williamson_county= TN_insured_data.query("County == 'Williamson'")
williamson_county.reset_index()
williamson_county

,Insured,Not_Insured,County,Year,Percent_Insured,Percent_Uninsured
93,149970,13859,Williamson,2010,91.54,8.46
188,151292,15370,Williamson,2011,90.78,9.22
283,149970,13859,Williamson,2010,91.54,8.46
378,151292,15370,Williamson,2011,90.78,9.22
473,156310,13906,Williamson,2012,91.83,8.17
568,159358,15523,Williamson,2013,91.12,8.88
663,166949,13645,Williamson,2014,92.44,7.56
758,175398,11184,Williamson,2015,94.01,5.99
853,182950,11006,Williamson,2016,94.33,5.67
948,188078,12631,Williamson,2017,93.71,6.29
